In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import wilcoxon
import scikit_posthocs as sp
import math 
from array import *

import warnings

SEED = 301
np.random.seed(SEED)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

warnings.filterwarnings('ignore')

In [2]:
resultado_final = pd.read_excel('Banco de dados Neurofat SEGUNDA ABA Imputado_ajustado1000kcal.xlsx'
                             ,sheet_name=1).drop(columns={"Unnamed: 0"})
resultado_final.head()

,GROUP,V0_QTD_G,V2_QTD_G,V4_QTD_G,V5_QTD_G,V0_CAL_KCAL,V2_CAL_KCAL,V4_CAL_KCAL,V5_CAL_KCAL,V0_PROT_G,V2_PROT_G,V4_PROT_G,V5_PROT_G,V0_CARB_G,V2_CARB_G,V4_CARB_G,V5_CARB_G,V0_FIB_G,V2_FIB_G,V4_FIB_G,V5_FIB_G,V0_SUGAR_G,V2_SUGAR_G,V4_SUGAR_G,V5_SUGAR_G,V0_FAT_G,V2_FAT_G,V4_FAT_G,V5_FAT_G,V0_SATFAT_G,V2_SATFAT_G,V4_SATFAT_G,V5_SATFAT_G,V0_MONOFAT_G,V2_MONOFAT_G,V4_MONOFAT_G,V5_MONOFAT_G,V0_POLIFAT_G,V2_POLIFAT_G,V4_POLIFAT_G,V5_POLIFAT_G,V0_CHOL_MG,V2_CHOL_MG,V4_CHOL_MG,V5_CHOL_MG,V0_18:2/18:3,V2_18:2/18:3,V4_18:2/18:3,V5_18:2/18:3,V0_12:0_G,V2_12:0_G,V4_12:0_G,V5_12:0_G,V0_14:0_G,V2_14:0_G,V4_14:0_G,V5_14:0_G,V0_16:0_G,V2_16:0_G,V4_16:0_G,V5_16:0_G,V0_18:0_G,V2_18:0_G,V4_18:0_G,V5_18:0_G,V0_18:1_G,V2_18:1_G,V4_18:1_G,V5_18:1_G,V0_18:2_G,V2_18:2_G,V4_18:2_G,V5_18:2_G,V0_18:3_G,V2_18:3_G,V4_18:3_G,V5_18:3_G,V0_20:4_G,V2_20:4_G,V4_20:4_G,V5_20:4_G,V0_20:5_G,V2_20:5_G,V4_20:5_G,V5_20:5_G,V0_22:6_G,V2_22:6_G,V4_22:6_G,V5_22:6_G,V0_IRON_MG,V2_IRON_MG,V4_IRON_MG,V5_IRON_MG,V0_ZINC_MG,V2_ZINC_MG,V4_ZINC_MG,V5_ZINC_MG,V0_SEL_MCG,V2_SEL_MCG,V4_SEL_MCG,V5_SEL_MCG,V0_SOD_MG,V2_SOD_MG,V4_SOD_MG,V5_SOD_MG,V0_VIT_ A_IU,V2_VIT_ A_IU,V4_VIT_ A_IU,V5_VIT_ A_IU,V0_VIT_ B6_MG,V2_VIT_ B6_MG,V4_VIT_ B6_MG,V5_VIT_ B6_MG,V0_VIT_B12_MCG,V2_VIT_B12_MCG,V4_VIT_B12_MCG,V5_VIT_B12_MCG,V0_FOL_ACID_MCG,V2_FOL_ACID_MCG,V4_FOL_ACID_MCG,V5_FOL_ACID_MCG,V0_VIT_D_IU,V2_VIT_D_IU,V4_VIT_D_IU,V5_VIT_D_IU,V0_VIT_E_IU,V2_VIT_E_IU,V4_VIT_E_IU,V5_VIT_E_IU,V0_VIT_C_MG,V2_VIT_C_MG,V4_VIT_C_MG,V5_VIT_C_MG
0,1,1196.287384,879.858152,1077.949833,814.222222,1000.0,1000.0,1000.0,1000.0,66.643749,91.562256,101.795958,116.693333,85.436003,40.613104,38.768700,27.182222,3.598029,12.722582,11.182558,5.457778,41.572132,15.218360,11.679352,19.040000,44.253466,56.722212,52.210266,46.951111,12.696230,8.436904,8.614225,10.648889,14.117108,12.646953,5.577219,24.568889,8.066919,19.554458,20.096734,3.022222,986.249570,395.349619,440.993963,342.648889,15.050000,18.291667,1.444444,7.833333,0.068752,0.008403,0.000000,0.000000,0.297926,0.008403,0.000000,0.391111,6.875215,1.016798,0.065614,6.062222,2.910508,0.210082,0.018747,2.328889,13.521256,8.193208,0.056241,23.573333,6.898132,3.689045,0.121855,2.506667,0.458348,0.201679,0.084361,0.320000,0.435430,0.000000,0.000000,0.088889,0.000000,0.000000,0.000000,0.000000,0.137504,0.000000,0.000000,0.000000,5.798098,2.941152,1.968430,6.862222,4.445972,2.050403,1.331034,11.288889,83.763034,3.529382,10.517041,69.137778,1607.081471,1951.462593,2191.406396,1062.168889,5135.510485,18514.264586,1477.709872,1572.266667,0.802108,0.487391,0.553035,1.226667,2.360490,0.000000,0.093735,3.591111,0.000000,0.000000,0.000000,0.000000,222.756961,0.000000,0.000000,10.666667,5.843933,13.268796,1.077950,6.648889,48.332760,95.016008,117.421544,90.542222
1,2,862.072531,794.909141,794.940644,718.127702,1000.0,1000.0,1000.0,1000.0,32.310595,64.310045,87.761447,70.965153,131.868586,42.839969,15.801653,22.534218,6.581022,6.673757,5.776569,7.898529,45.581203,26.379063,9.362634,12.203501,40.235738,65.396649,64.734666,70.952992,11.373330,18.364390,19.555540,17.998188,4.937059,16.283658,12.639556,16.648324,0.863339,12.599991,9.574618,14.319504,64.378215,427.251430,800.584723,751.035200,5.500000,0.841505,7.301887,1.537015,0.046527,0.377614,0.423968,0.304023,0.356709,1.209907,1.581049,1.222174,0.940885,5.795225,6.880653,5.940618,0.351539,2.589356,3.109101,2.523395,0.863339,15.158521,9.645280,14.435033,0.398067,5.687335,6.836490,7.953253,0.072376,6.758527,0.936263,5.174479,0.000000,0.007706,0.105992,0.030402,0.000000,0.000000,0.697781,0.486438,0.000000,0.000000,0.759610,0.522920,4.420089,2.843668,9.185981,9.844279,3.189702,3.082567,6.907151,6.195998,30.335772,20.329526,129.010034,79.508212,1116.829943,2011.644395,1320.131430,1068.727540,2508.413679,3318.344354,3652.257631,3707.851710,0.113733,0.516330,1.254240,0.833024,1.075297,0.747522,14.141111,10.379360,0.000000,0.000000,0.000000,0.000000,2.140254,8.322930,342.354438,207.733140,0.170600,11.027882,8.011235,13.541204,67.500711,63.955549

In [3]:
resultado_final.query('GROUP == 1').head()

,GROUP,V0_QTD_G,V2_QTD_G,V4_QTD_G,V5_QTD_G,V0_CAL_KCAL,V2_CAL_KCAL,V4_CAL_KCAL,V5_CAL_KCAL,V0_PROT_G,V2_PROT_G,V4_PROT_G,V5_PROT_G,V0_CARB_G,V2_CARB_G,V4_CARB_G,V5_CARB_G,V0_FIB_G,V2_FIB_G,V4_FIB_G,V5_FIB_G,V0_SUGAR_G,V2_SUGAR_G,V4_SUGAR_G,V5_SUGAR_G,V0_FAT_G,V2_FAT_G,V4_FAT_G,V5_FAT_G,V0_SATFAT_G,V2_SATFAT_G,V4_SATFAT_G,V5_SATFAT_G,V0_MONOFAT_G,V2_MONOFAT_G,V4_MONOFAT_G,V5_MONOFAT_G,V0_POLIFAT_G,V2_POLIFAT_G,V4_POLIFAT_G,V5_POLIFAT_G,V0_CHOL_MG,V2_CHOL_MG,V4_CHOL_MG,V5_CHOL_MG,V0_18:2/18:3,V2_18:2/18:3,V4_18:2/18:3,V5_18:2/18:3,V0_12:0_G,V2_12:0_G,V4_12:0_G,V5_12:0_G,V0_14:0_G,V2_14:0_G,V4_14:0_G,V5_14:0_G,V0_16:0_G,V2_16:0_G,V4_16:0_G,V5_16:0_G,V0_18:0_G,V2_18:0_G,V4_18:0_G,V5_18:0_G,V0_18:1_G,V2_18:1_G,V4_18:1_G,V5_18:1_G,V0_18:2_G,V2_18:2_G,V4_18:2_G,V5_18:2_G,V0_18:3_G,V2_18:3_G,V4_18:3_G,V5_18:3_G,V0_20:4_G,V2_20:4_G,V4_20:4_G,V5_20:4_G,V0_20:5_G,V2_20:5_G,V4_20:5_G,V5_20:5_G,V0_22:6_G,V2_22:6_G,V4_22:6_G,V5_22:6_G,V0_IRON_MG,V2_IRON_MG,V4_IRON_MG,V5_IRON_MG,V0_ZINC_MG,V2_ZINC_MG,V4_ZINC_MG,V5_ZINC_MG,V0_SEL_MCG,V2_SEL_MCG,V4_SEL_MCG,V5_SEL_MCG,V0_SOD_MG,V2_SOD_MG,V4_SOD_MG,V5_SOD_MG,V0_VIT_ A_IU,V2_VIT_ A_IU,V4_VIT_ A_IU,V5_VIT_ A_IU,V0_VIT_ B6_MG,V2_VIT_ B6_MG,V4_VIT_ B6_MG,V5_VIT_ B6_MG,V0_VIT_B12_MCG,V2_VIT_B12_MCG,V4_VIT_B12_MCG,V5_VIT_B12_MCG,V0_FOL_ACID_MCG,V2_FOL_ACID_MCG,V4_FOL_ACID_MCG,V5_FOL_ACID_MCG,V0_VIT_D_IU,V2_VIT_D_IU,V4_VIT_D_IU,V5_VIT_D_IU,V0_VIT_E_IU,V2_VIT_E_IU,V4_VIT_E_IU,V5_VIT_E_IU,V0_VIT_C_MG,V2_VIT_C_MG,V4_VIT_C_MG,V5_VIT_C_MG
0,1,1196.287384,879.858152,1077.949833,814.222222,1000.0,1000.0,1000.0,1000.0,66.643749,91.562256,101.795958,116.693333,85.436003,40.613104,38.768700,27.182222,3.598029,12.722582,11.182558,5.457778,41.572132,15.218360,11.679352,19.040000,44.253466,56.722212,52.210266,46.951111,12.696230,8.436904,8.614225,10.648889,14.117108,12.646953,5.577219,24.568889,8.066919,19.554458,20.096734,3.022222,986.249570,395.349619,440.993963,342.648889,15.050000,18.291667,1.444444,7.833333,0.068752,0.008403,0.000000,0.000000,0.297926,0.008403,0.000000,0.391111,6.875215,1.016798,0.065614,6.062222,2.910508,0.210082,0.018747,2.328889,13.521256,8.193208,0.056241,23.573333,6.898132,3.689045,0.121855,2.506667,0.458348,0.201679,0.084361,0.320000,0.435430,0.000000,0.000000,0.088889,0.0,0.0,0.0,0.0,0.137504,0.0,0.0,0.000000,5.798098,2.941152,1.968430,6.862222,4.445972,2.050403,1.331034,11.288889,83.763034,3.529382,10.517041,69.137778,1607.081471,1951.462593,2191.406396,1062.168889,5135.510485,18514.264586,1477.709872,1572.266667,0.802108,0.487391,0.553035,1.226667,2.360490,0.000000,0.093735,3.591111,0.000000,0.0,0.0,0.0,222.756961,0.000000,0.000000,10.666667,5.843933,13.268796,1.077950,6.648889,48.332760,95.016008,117.421544,90.542222
3,1,709.154674,593.279048,1199.900685,909.821572,1000.0,1000.0,1000.0,1000.0,36.618696,69.350678,129.625583,111.950562,115.951143,46.350871,79.014798,49.206094,8.731406,17.820018,17.598570,6.965654,30.052001,10.878900,50.134075,19.896765,45.591970,60.636465,21.293078,40.382932,10.799371,8.436904,5.919158,7.269285,21.260128,19.041857,3.416427,19.432389,6.119241,19.504038,0.675340,2.625516,456.367154,167.795229,390.128116,341.995749,15.206897,4.299706,3.571429,7.937500,0.000000,0.028571,0.000000,0.000000,0.060467,0.354619,0.198629,0.000000,5.708066,7.142797,1.747939,2.893425,2.055871,2.243679,0.834244,0.696565,20.643367,4.850379,3.098620,19.039454,5.333172,3.689045,0.496574,2.268303,0.350707,0.857976,0.139041,0.285770,0.229774,0.009244,0.039726,0.000000,0.0,0.0,0.0,0.0,0.072560,0.0,0.0,0.000000,5.877373,2.941152,4.866422,2.536212,2.890313,9.420089,6.852716,1.446713,66.271617,5.471383,39.110140,7.447892,1297.520861,1544.891219,1761.128215,2410.902141,1733.389769,337.249267,2179.203496,1703.013092,0.689322,0.433610,0.794518,0.339352,1.076309,0.000000,1.886980,0.000000,106.119241,0.0,0.0,0.0,97.907849,1.848724,5.025325,0.000000,6.421575,6.651205,2.999305,5.965458,15.298101,44.867690,43.062866,23.665363
6,1,848.785100,588.320211,770.776887,874.048137,1000.0,1000.0,1000.0,1000.

In [4]:
def MudarFormatacaoDados(dados):
    lista = list()
    for item in dados.columns:
        item = item.replace("(","")
        item = item.replace(")","")
        item = item.replace(":","_")
        item = item.replace(" ","")
        item = item.replace("/","-")
        lista.append(item)
    dados.columns = lista
    return dados

In [5]:
resultado_final = MudarFormatacaoDados(resultado_final).replace("--",float("NAN"))

In [6]:
resultado_final.head()

,GROUP,V0_QTD_G,V2_QTD_G,V4_QTD_G,V5_QTD_G,V0_CAL_KCAL,V2_CAL_KCAL,V4_CAL_KCAL,V5_CAL_KCAL,V0_PROT_G,V2_PROT_G,V4_PROT_G,V5_PROT_G,V0_CARB_G,V2_CARB_G,V4_CARB_G,V5_CARB_G,V0_FIB_G,V2_FIB_G,V4_FIB_G,V5_FIB_G,V0_SUGAR_G,V2_SUGAR_G,V4_SUGAR_G,V5_SUGAR_G,V0_FAT_G,V2_FAT_G,V4_FAT_G,V5_FAT_G,V0_SATFAT_G,V2_SATFAT_G,V4_SATFAT_G,V5_SATFAT_G,V0_MONOFAT_G,V2_MONOFAT_G,V4_MONOFAT_G,V5_MONOFAT_G,V0_POLIFAT_G,V2_POLIFAT_G,V4_POLIFAT_G,V5_POLIFAT_G,V0_CHOL_MG,V2_CHOL_MG,V4_CHOL_MG,V5_CHOL_MG,V0_18_2-18_3,V2_18_2-18_3,V4_18_2-18_3,V5_18_2-18_3,V0_12_0_G,V2_12_0_G,V4_12_0_G,V5_12_0_G,V0_14_0_G,V2_14_0_G,V4_14_0_G,V5_14_0_G,V0_16_0_G,V2_16_0_G,V4_16_0_G,V5_16_0_G,V0_18_0_G,V2_18_0_G,V4_18_0_G,V5_18_0_G,V0_18_1_G,V2_18_1_G,V4_18_1_G,V5_18_1_G,V0_18_2_G,V2_18_2_G,V4_18_2_G,V5_18_2_G,V0_18_3_G,V2_18_3_G,V4_18_3_G,V5_18_3_G,V0_20_4_G,V2_20_4_G,V4_20_4_G,V5_20_4_G,V0_20_5_G,V2_20_5_G,V4_20_5_G,V5_20_5_G,V0_22_6_G,V2_22_6_G,V4_22_6_G,V5_22_6_G,V0_IRON_MG,V2_IRON_MG,V4_IRON_MG,V5_IRON_MG,V0_ZINC_MG,V2_ZINC_MG,V4_ZINC_MG,V5_ZINC_MG,V0_SEL_MCG,V2_SEL_MCG,V4_SEL_MCG,V5_SEL_MCG,V0_SOD_MG,V2_SOD_MG,V4_SOD_MG,V5_SOD_MG,V0_VIT_A_IU,V2_VIT_A_IU,V4_VIT_A_IU,V5_VIT_A_IU,V0_VIT_B6_MG,V2_VIT_B6_MG,V4_VIT_B6_MG,V5_VIT_B6_MG,V0_VIT_B12_MCG,V2_VIT_B12_MCG,V4_VIT_B12_MCG,V5_VIT_B12_MCG,V0_FOL_ACID_MCG,V2_FOL_ACID_MCG,V4_FOL_ACID_MCG,V5_FOL_ACID_MCG,V0_VIT_D_IU,V2_VIT_D_IU,V4_VIT_D_IU,V5_VIT_D_IU,V0_VIT_E_IU,V2_VIT_E_IU,V4_VIT_E_IU,V5_VIT_E_IU,V0_VIT_C_MG,V2_VIT_C_MG,V4_VIT_C_MG,V5_VIT_C_MG
0,1,1196.287384,879.858152,1077.949833,814.222222,1000.0,1000.0,1000.0,1000.0,66.643749,91.562256,101.795958,116.693333,85.436003,40.613104,38.768700,27.182222,3.598029,12.722582,11.182558,5.457778,41.572132,15.218360,11.679352,19.040000,44.253466,56.722212,52.210266,46.951111,12.696230,8.436904,8.614225,10.648889,14.117108,12.646953,5.577219,24.568889,8.066919,19.554458,20.096734,3.022222,986.249570,395.349619,440.993963,342.648889,15.050000,18.291667,1.444444,7.833333,0.068752,0.008403,0.000000,0.000000,0.297926,0.008403,0.000000,0.391111,6.875215,1.016798,0.065614,6.062222,2.910508,0.210082,0.018747,2.328889,13.521256,8.193208,0.056241,23.573333,6.898132,3.689045,0.121855,2.506667,0.458348,0.201679,0.084361,0.320000,0.435430,0.000000,0.000000,0.088889,0.000000,0.000000,0.000000,0.000000,0.137504,0.000000,0.000000,0.000000,5.798098,2.941152,1.968430,6.862222,4.445972,2.050403,1.331034,11.288889,83.763034,3.529382,10.517041,69.137778,1607.081471,1951.462593,2191.406396,1062.168889,5135.510485,18514.264586,1477.709872,1572.266667,0.802108,0.487391,0.553035,1.226667,2.360490,0.000000,0.093735,3.591111,0.000000,0.000000,0.000000,0.000000,222.756961,0.000000,0.000000,10.666667,5.843933,13.268796,1.077950,6.648889,48.332760,95.016008,117.421544,90.542222
1,2,862.072531,794.909141,794.940644,718.127702,1000.0,1000.0,1000.0,1000.0,32.310595,64.310045,87.761447,70.965153,131.868586,42.839969,15.801653,22.534218,6.581022,6.673757,5.776569,7.898529,45.581203,26.379063,9.362634,12.203501,40.235738,65.396649,64.734666,70.952992,11.373330,18.364390,19.555540,17.998188,4.937059,16.283658,12.639556,16.648324,0.863339,12.599991,9.574618,14.319504,64.378215,427.251430,800.584723,751.035200,5.500000,0.841505,7.301887,1.537015,0.046527,0.377614,0.423968,0.304023,0.356709,1.209907,1.581049,1.222174,0.940885,5.795225,6.880653,5.940618,0.351539,2.589356,3.109101,2.523395,0.863339,15.158521,9.645280,14.435033,0.398067,5.687335,6.836490,7.953253,0.072376,6.758527,0.936263,5.174479,0.000000,0.007706,0.105992,0.030402,0.000000,0.000000,0.697781,0.486438,0.000000,0.000000,0.759610,0.522920,4.420089,2.843668,9.185981,9.844279,3.189702,3.082567,6.907151,6.195998,30.335772,20.329526,129.010034,79.508212,1116.829943,2011.644395,1320.131430,1068.727540,2508.413679,3318.344354,3652.257631,3707.851710,0.113733,0.516330,1.254240,0.833024,1.075297,0.747522,14.141111,10.379360,0.000000,0.000000,0.000000,0.000000,2.140254,8.322930,342.354438,207.733140,0.170600,11.027882,8.011235,13.541204,67.500711,63.955549,21.5958

In [7]:
def ListaDeParametros(dados_brutos):
    lista = list()
    for column in dados_brutos.columns:
        if "V0" in column:
            lista.append(column.replace("V0",""))
            
    return pd.Series(lista)

In [8]:
parametros = ListaDeParametros(resultado_final)
parametros

0            _QTD_G
1         _CAL_KCAL
2           _PROT_G
3           _CARB_G
4            _FIB_G
5          _SUGAR_G
6            _FAT_G
7         _SATFAT_G
8        _MONOFAT_G
9        _POLIFAT_G
10         _CHOL_MG
11       _18_2-18_3
12          _12_0_G
13          _14_0_G
14          _16_0_G
15          _18_0_G
16          _18_1_G
17          _18_2_G
18          _18_3_G
19          _20_4_G
20          _20_5_G
21          _22_6_G
22         _IRON_MG
23         _ZINC_MG
24         _SEL_MCG
25          _SOD_MG
26        _VIT_A_IU
27       _VIT_B6_MG
28     _VIT_B12_MCG
29    _FOL_ACID_MCG
30        _VIT_D_IU
31        _VIT_E_IU
32        _VIT_C_MG
dtype: object

In [9]:
def GerandoResultados(parametros,dados):
    tabela_resultados = pd.DataFrame(columns=['V0','V5','p','Delta Média','Paramétrico'])
    colunaVariacao = pd.DataFrame()
    
    for item in parametros:
        
        mediaInicial = dados["V0"+item].mean()
        desvioPadraoInicial = dados["V0"+item].std()
        v0 = str(round(mediaInicial,1))+" ("+str(round(desvioPadraoInicial,1))+")"
        mediaFinal = dados["V5"+item].mean()
        desvioPadraoFinal = dados["V5"+item].std()
        v5 = str(round(mediaFinal,1))+" ("+str(round(desvioPadraoFinal,1))+")"
    
        delta_media= round(mediaFinal-mediaInicial,1)
    
        colVariacao = dados["V5"+item]-dados["V0"+item]
    
        t_ks,p_ks = stats.kstest(colVariacao, 'norm')
    
        p=0
        parametrico=""
    
        if p_ks>=0.05:
            t,p = stats.ttest_rel(a=dados["V0"+item],b=dados["V5"+item])
            p = round(p,3)
            parametrico="sim"
        else:
            w, p = wilcoxon(x=dados["V0"+item],y=dados["V5"+item],zero_method='zsplit')
            p = round(p,3)
            parametrico="nao"
    
        tabela_resultados = tabela_resultados.append({'V0': v0, 'V5': v5, 'p': p,'Delta Média':delta_media,
                                                        'Paramétrico':parametrico}, ignore_index=True)
        #print(pretabela_resultados)
        #print(colunaVariacao)
        colunaVariacao[item]=colVariacao
        
    return tabela_resultados,colunaVariacao

In [10]:
studant1,colunaVariacao1=GerandoResultados(parametros,resultado_final.query('GROUP == 1'))

In [11]:
studant1.index = parametros
studant1

,V0,V5,p,Delta Média,Paramétrico
_QTD_G,704.7 (294.1),802.5 (242.6),0.263,97.8,nao
_CAL_KCAL,1000.0 (0.0),1000.0 (0.0),0.271,0.0,nao
_PROT_G,50.6 (24.8),90.7 (17.8),0.025,40.1,nao
_CARB_G,121.5 (19.9),42.6 (9.8),0.012,-78.9,nao
_FIB_G,8.6 (4.5),7.7 (3.0),0.516,-0.8,sim
_SUGAR_G,30.9 (21.0),18.6 (6.8),0.050,-12.3,nao
_FAT_G,36.3 (10.8),57.5 (13.0),0.025,21.2,nao
_SATFAT_G,10.7 (4.3),12.1 (2.9),0.263,1.4,nao
_MONOFAT_G,10.0 (6.3),17.0 (5.3),0.025,7.0,nao
_POLIFAT_G,5.0 (4.4),12.2 (15.4),0.575,7.2,nao


In [12]:
studant1.to_excel("Dados_extras_tabela_studant1.xlsx")

In [13]:
studant2,colunaVariacao2=GerandoResultados(parametros,resultado_final.query('GROUP == 2'))

In [14]:
studant2.index = parametros
studant2

,V0,V5,p,Delta Média,Paramétrico
_QTD_G,702.5 (108.5),929.5 (369.9),0.237,227.1,nao
_CAL_KCAL,1000.0 (0.0),1000.0 (0.0),0.119,0.0,nao
_PROT_G,38.7 (15.7),80.3 (23.3),0.018,41.6,nao
_CARB_G,128.9 (13.8),32.5 (13.7),0.018,-96.4,nao
_FIB_G,7.8 (3.1),7.9 (3.5),0.979,0.1,sim
_SUGAR_G,39.5 (19.6),13.5 (6.3),0.018,-26.0,nao
_FAT_G,38.3 (5.3),68.5 (15.1),0.018,30.2,nao
_SATFAT_G,13.6 (5.4),25.9 (14.9),0.176,12.3,nao
_MONOFAT_G,5.3 (4.1),24.3 (9.2),0.018,19.0,nao
_POLIFAT_G,2.1 (1.5),15.5 (9.2),0.028,13.4,nao


In [15]:
studant2.to_excel("Dados_extras_tabela_studant2.xlsx")

In [16]:
studant3,colunaVariacao3=GerandoResultados(parametros,resultado_final.query('GROUP == 3'))

In [17]:
studant3.index = parametros
studant3

,V0,V5,p,Delta Média,Paramétrico
_QTD_G,759.3 (234.3),899.4 (341.7),0.237,140.1,nao
_CAL_KCAL,1000.0 (0.0),1000.0 (0.0),0.603,0.0,nao
_PROT_G,42.8 (24.7),44.0 (9.2),0.735,1.2,nao
_CARB_G,126.6 (31.1),134.9 (24.4),0.499,8.3,nao
_FIB_G,10.6 (3.0),11.6 (4.7),0.666,1.0,sim
_SUGAR_G,53.5 (29.9),67.1 (19.8),0.398,13.7,nao
_FAT_G,43.4 (16.3),43.2 (17.4),0.612,-0.1,nao
_SATFAT_G,13.3 (7.5),15.2 (4.6),0.499,1.9,nao
_MONOFAT_G,8.9 (5.5),9.1 (5.0),0.735,0.2,nao
_POLIFAT_G,4.5 (2.2),3.1 (1.4),0.310,-1.4,nao


In [18]:
studant3.to_excel("Dados_extras_tabela_studant3.xlsx")

In [19]:
def AnovaMedia(parametros,studant):

    tabelaNova_media = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(colunaVariacao1[i],
                                  colunaVariacao2[i],
                                  colunaVariacao3[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            F, p = stats.kruskal(colunaVariacao1[i],
                                 colunaVariacao2[i],
                                 colunaVariacao3[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        
        x=x+1
        
    tabelaNova_media.index = parametros

    return tabelaNova_media

In [20]:
tabela_Anova_Media = AnovaMedia(parametros,studant1)
tabela_Anova_Media

,p,F
_QTD_G,0.752,0.570440
_CAL_KCAL,0.093,4.743228
_PROT_G,0.029,7.089215
_CARB_G,0.001,13.495342
_FIB_G,0.790,0.238442
_SUGAR_G,0.118,4.276821
_FAT_G,0.037,6.601355
_SATFAT_G,0.231,2.933936
_MONOFAT_G,0.007,9.900762
_POLIFAT_G,0.045,6.194805


In [21]:
tabela_Anova_Media.to_excel("Dados_extras_tabela_Anova_media.xlsx")

In [22]:
def Anova(parametros,studant,dados):

    dado1 = dados.query('GROUP == 1')
    dado2 = dados.query('GROUP == 2')
    dado3 = dados.query('GROUP == 3')
    
    tabelaNova = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(dado1[i],
                                  dado2[i],
                                  dado3[i]) 
            tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            try:
                F, p = stats.kruskal(dado1[i],
                                     dado2[i],
                                     dado3[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
            except:
                F, p = stats.f_oneway(dado1[i],
                                      dado2[i],
                                      dado3[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
                
        x=x+1
        
    tabelaNova.index = parametros

    return tabelaNova

In [23]:
tabela_Anova_Inicial = Anova("V0"+parametros,studant1,resultado_final)
tabela_Anova_Inicial

,p,F
V0_QTD_G,0.720,0.657256
V0_CAL_KCAL,0.249,2.783487
V0_PROT_G,0.700,0.713156
V0_CARB_G,0.804,0.435347
V0_FIB_G,0.368,1.054189
V0_SUGAR_G,0.303,2.386364
V0_FAT_G,0.882,0.250706
V0_SATFAT_G,0.426,1.708780
V0_MONOFAT_G,0.244,2.820440
V0_POLIFAT_G,0.180,3.430265


In [24]:
tabela_Anova_Inicial.to_excel("Dados_extras_tabela_anova_inicial.xlsx")

In [25]:
tabela_Anova_Final = Anova("V5"+parametros,studant1,resultado_final)
tabela_Anova_Final

,p,F
V5_QTD_G,0.752,0.570763
V5_CAL_KCAL,0.277,2.566667
V5_PROT_G,0.001,13.450028
V5_CARB_G,0.001,14.626908
V5_FIB_G,0.118,2.393279
V5_SUGAR_G,0.001,14.610390
V5_FAT_G,0.031,6.962769
V5_SATFAT_G,0.032,6.860781
V5_MONOFAT_G,0.003,11.967815
V5_POLIFAT_G,0.030,7.025424


In [26]:
tabela_Anova_Final.to_excel("Dados_extras_tabela_anova_final.xlsx")

# Grupo 1 e 2

In [27]:
def AnovaMedia1e2(parametros,studant):

    tabelaNova_media = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(colunaVariacao1[i],
                                  colunaVariacao2[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            F, p = stats.kruskal(colunaVariacao1[i],
                                 colunaVariacao2[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        
        x=x+1
        
    tabelaNova_media.index = parametros

    return tabelaNova_media

In [28]:
def Anova1e2(parametros,studant,dados):

    dado1 = dados.query('GROUP == 1')
    dado2 = dados.query('GROUP == 2')
    
    tabelaNova = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(dado1[i],
                                  dado2[i]) 
            tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            try:
                F, p = stats.kruskal(dado1[i],
                                     dado2[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
            except:
                F, p = stats.f_oneway(dado1[i],
                                      dado2[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
                
        x=x+1
        
    tabelaNova.index = parametros

    return tabelaNova

In [29]:
tabela_Anova_Media1e2 = AnovaMedia1e2(parametros,studant1)
tabela_Anova_Media1e2

,p,F
_QTD_G,0.563,0.334821
_CAL_KCAL,0.027,4.923077
_PROT_G,0.643,0.214286
_CARB_G,0.064,3.428571
_FIB_G,0.724,0.129953
_SUGAR_G,0.298,1.084821
_FAT_G,0.298,1.084821
_SATFAT_G,0.105,2.625000
_MONOFAT_G,0.028,4.834821
_POLIFAT_G,0.165,1.928571


In [30]:
tabela_Anova_Media1e2.to_excel("Dados_extras_tabela_Anova_media1e2.xlsx")

In [31]:
tabela_Anova_Inicial1e2 = Anova1e2("V0"+parametros,studant1,resultado_final)
tabela_Anova_Inicial1e2

,p,F
V0_QTD_G,0.908,0.013393
V0_CAL_KCAL,0.082,3.028846
V0_PROT_G,0.487,0.482143
V0_CARB_G,0.487,0.482143
V0_FIB_G,0.725,0.129444
V0_SUGAR_G,0.355,0.857143
V0_FAT_G,0.908,0.013393
V0_SATFAT_G,0.203,1.620536
V0_MONOFAT_G,0.132,2.263393
V0_POLIFAT_G,0.203,1.620536


In [32]:
tabela_Anova_Inicial1e2.to_excel("Dados_extras_tabela_anova_inicial1e2.xlsx")

In [33]:
tabela_Anova_Final1e2 = Anova1e2("V5"+parametros,studant1,resultado_final)
tabela_Anova_Final1e2

,p,F
V5_QTD_G,0.643,0.214669
V5_CAL_KCAL,0.171,1.875000
V5_PROT_G,0.247,1.339286
V5_CARB_G,0.164,1.935484
V5_FIB_G,0.930,0.007987
V5_SUGAR_G,0.165,1.928571
V5_FAT_G,0.163,1.945946
V5_SATFAT_G,0.015,5.948741
V5_MONOFAT_G,0.083,3.013393
V5_POLIFAT_G,0.165,1.932021


In [34]:
tabela_Anova_Final1e2.to_excel("Dados_extras_tabela_anova_final1e2.xlsx")

# Grupos 1 e 3

In [35]:
def AnovaMedia1e3(parametros,studant):

    tabelaNova_media = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(colunaVariacao1[i],
                                  colunaVariacao3[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            F, p = stats.kruskal(colunaVariacao1[i],
                                 colunaVariacao3[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        
        x=x+1
        
    tabelaNova_media.index = parametros

    return tabelaNova_media

In [36]:
def Anova1e3(parametros,studant,dados):

    dado1 = dados.query('GROUP == 1')
    dado2 = dados.query('GROUP == 3')
    
    tabelaNova = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(dado1[i],
                                  dado2[i]) 
            tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            try:
                F, p = stats.kruskal(dado1[i],
                                     dado2[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
            except:
                F, p = stats.f_oneway(dado1[i],
                                      dado2[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
                
        x=x+1
        
    tabelaNova.index = parametros

    return tabelaNova

In [37]:
tabela_Anova_Media1e3 = AnovaMedia1e3(parametros,studant1)
tabela_Anova_Media1e3

,p,F
_QTD_G,0.487,0.482143
_CAL_KCAL,0.212,1.557692
_PROT_G,0.021,5.357143
_CARB_G,0.004,8.370536
_FIB_G,0.466,0.564604
_SUGAR_G,0.165,1.928571
_FAT_G,0.049,3.870536
_SATFAT_G,0.643,0.214286
_MONOFAT_G,0.105,2.625000
_POLIFAT_G,0.643,0.214286


In [38]:
tabela_Anova_Media1e3.to_excel("Dados_extras_tabela_Anova_media1e3.xlsx")

In [39]:
tabela_Anova_Inicial1e3 = Anova1e3("V0"+parametros,studant1,resultado_final)
tabela_Anova_Inicial1e3

,p,F
V0_QTD_G,0.563,0.334821
V0_CAL_KCAL,0.619,0.247253
V0_PROT_G,0.487,0.482143
V0_CARB_G,0.817,0.053571
V0_FIB_G,0.336,0.996203
V0_SUGAR_G,0.132,2.263393
V0_FAT_G,0.418,0.656250
V0_SATFAT_G,0.355,0.857143
V0_MONOFAT_G,0.728,0.120536
V0_POLIFAT_G,0.728,0.120536


In [40]:
tabela_Anova_Inicial1e3.to_excel("Dados_extras_tabela_anova_inicial1e3.xlsx")

In [41]:
tabela_Anova_Final1e3 = Anova1e3("V5"+parametros,studant1,resultado_final)
tabela_Anova_Final1e3

,p,F
V5_QTD_G,0.418,0.656250
V5_CAL_KCAL,0.171,1.875000
V5_PROT_G,0.001,10.500000
V5_CARB_G,0.001,10.518784
V5_FIB_G,0.078,3.661458
V5_SUGAR_G,0.001,10.500000
V5_FAT_G,0.131,2.279676
V5_SATFAT_G,0.201,1.632194
V5_MONOFAT_G,0.008,7.084821
V5_POLIFAT_G,0.643,0.214669


In [42]:
tabela_Anova_Final1e3.to_excel("Dados_extras_tabela_anova_final1e3.xlsx")

# Grupos 2 e 3

In [43]:
def AnovaMedia2e3(parametros,studant):

    tabelaNova_media = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(colunaVariacao2[i],
                                  colunaVariacao3[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            F, p = stats.kruskal(colunaVariacao2[i],
                                 colunaVariacao3[i]) 
            tabelaNova_media = tabelaNova_media.append({'p': round(p,3), 'F': F}, ignore_index=True)
        
        x=x+1
        
    tabelaNova_media.index = parametros

    return tabelaNova_media

In [44]:
def Anova2e3(parametros,studant,dados):

    dado1 = dados.query('GROUP == 2')
    dado2 = dados.query('GROUP == 3')
    
    tabelaNova = pd.DataFrame(columns=['p','F'])
    x=0
    for i in parametros: 
        if studant["Paramétrico"][x]=="sim":
            F, p = stats.f_oneway(dado1[i],
                                  dado2[i]) 
            tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
        else:
            try:
                F, p = stats.kruskal(dado1[i],
                                     dado2[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
            except:
                F, p = stats.f_oneway(dado1[i],
                                      dado2[i]) 
                tabelaNova = tabelaNova.append({'p': round(p,3), 'F': F}, ignore_index=True)
                
        x=x+1
        
    tabelaNova.index = parametros

    return tabelaNova

In [45]:
tabela_Anova_Media2e3 = AnovaMedia2e3(parametros,studant1)
tabela_Anova_Media2e3

,p,F
_QTD_G,0.848,0.036735
_CAL_KCAL,0.424,0.640212
_PROT_G,0.025,5.000000
_CARB_G,0.003,9.016327
_FIB_G,0.769,0.090219
_SUGAR_G,0.064,3.432653
_FAT_G,0.025,5.000000
_SATFAT_G,0.225,1.473469
_MONOFAT_G,0.006,7.546939
_POLIFAT_G,0.006,7.546939


In [46]:
tabela_Anova_Media2e3.to_excel("Dados_extras_tabela_Anova_media2e3.xlsx")

In [47]:
tabela_Anova_Inicial2e3 = Anova2e3("V0"+parametros,studant1,resultado_final)
tabela_Anova_Inicial2e3

,p,F
V0_QTD_G,0.406,0.689796
V0_CAL_KCAL,0.334,0.935065
V0_PROT_G,0.749,0.102041
V0_CARB_G,0.749,0.102041
V0_FIB_G,0.122,2.776322
V0_SUGAR_G,0.565,0.330612
V0_FAT_G,0.949,0.004082
V0_SATFAT_G,0.949,0.004082
V0_MONOFAT_G,0.180,1.800000
V0_POLIFAT_G,0.064,3.432653


In [48]:
tabela_Anova_Inicial2e3.to_excel("Dados_extras_tabela_anova_inicial2e3.xlsx")

In [49]:
tabela_Anova_Final2e3 = Anova2e3("V5"+parametros,studant1,resultado_final)
tabela_Anova_Final2e3

,p,F
V5_QTD_G,0.949,0.004091
V5_CAL_KCAL,1.000,0.000000
V5_PROT_G,0.004,8.265306
V5_CARB_G,0.002,9.821586
V5_FIB_G,0.122,2.762569
V5_SUGAR_G,0.002,9.800000
V5_FAT_G,0.013,6.221838
V5_SATFAT_G,0.110,2.551020
V5_MONOFAT_G,0.004,8.265306
V5_POLIFAT_G,0.003,9.016327


In [50]:
tabela_Anova_Final2e3.to_excel("Dados_extras_tabela_anova_final2e3.xlsx")

# Posthoc

In [51]:
def FazerPostHocMedia(variavel):
    tabelaPostHoc = pd.DataFrame()
    tabelaPostHoc["grupos"] = resultado_final['GROUP']

    #try:
    #    tabelaPostHoc[variavel] = resultado_final[variavel+"V_6"]-resultado_final[variavel+"V_0"]
    #except:
    #    print("Não tem v6")
    
    try:
        tabelaPostHoc[variavel] = resultado_final["V5"+variavel]-resultado_final["V0"+variavel]
    except:
        print("Não tem v5")
    
    
    tabela_posthoc_final = sp.posthoc_ttest(tabelaPostHoc,val_col=variavel,group_col='grupos',
                                            p_adjust='bonferroni')
    tabela_posthoc_final = tabela_posthoc_final.rename({1:variavel+"=>1"}, axis='columns')
    return tabela_posthoc_final

In [52]:
for i in parametros:
    p = tabela_Anova_Media.loc[i][0]
    if p < 0.05:
        print('{} p = {}'.format(i,p))
        posthoc = FazerPostHocMedia(i)
        posthoc.to_excel('Dados_extras_posthoc_'+i+'_media.xlsx')


_PROT_G p = 0.029
_CARB_G p = 0.001
_FAT_G p = 0.037
_MONOFAT_G p = 0.007
_POLIFAT_G p = 0.045
_18_2-18_3 p = 0.042
_12_0_G p = 0.017
_14_0_G p = 0.014
_16_0_G p = 0.007
_18_1_G p = 0.008
_18_2_G p = 0.023
_18_3_G p = 0.007
_20_5_G p = 0.002
_22_6_G p = 0.009
_IRON_MG p = 0.0
_SEL_MCG p = 0.001
_VIT_B6_MG p = 0.004
_VIT_B12_MCG p = 0.001
_FOL_ACID_MCG p = 0.008
_VIT_D_IU p = 0.022
_VIT_E_IU p = 0.007


In [53]:
def FazerPostHocInicial(variavel):
    tabelaPostHoc = pd.DataFrame()
    tabelaPostHoc["grupos"] = resultado_final['GROUP']

    #try:
    #    tabelaPostHoc[variavel] =resultado_final[variavel+"V0"]
    #except:
    #    print("Não tem v6")
    
    try:
        tabelaPostHoc[variavel] =resultado_final["V0"+variavel]
    except:
        print("Não tem v5")
    
    
    tabela_posthoc_final = sp.posthoc_ttest(tabelaPostHoc,val_col=variavel,group_col='grupos',p_adjust='bonferroni')
    tabela_posthoc_final = tabela_posthoc_final.rename({1:variavel+"=>1"}, axis='columns')
    return tabela_posthoc_final

In [54]:
for i in parametros:
    p = tabela_Anova_Inicial.loc["V0"+i][0]
    if p < 0.05:
        if math.isnan(p):
            print(i+' é nan')
        else:
            print('{} p = {}'.format(i,p))
            posthoc = FazerPostHocInicial(i)
            posthoc.to_excel('Dados_extras_posthoc_'+i+'_inicial.xlsx')


_IRON_MG p = 0.021
_FOL_ACID_MCG p = 0.047


In [55]:
def FazerPostHocFinal(variavel):
    tabelaPostHoc = pd.DataFrame()
    tabelaPostHoc["grupos"] = resultado_final['GROUP']

    #try:
    #    tabelaPostHoc[variavel] =resultado_final[variavel+"V6"]
    #except:
    #    print("Não tem v6")
    
    try:
        tabelaPostHoc[variavel] =resultado_final["V5"+variavel]
    except:
        print("Não tem v5")
    
    
    tabela_posthoc_final = sp.posthoc_ttest(tabelaPostHoc,val_col=variavel,group_col='grupos',p_adjust='bonferroni')
    tabela_posthoc_final = tabela_posthoc_final.rename({1:variavel+"=>1"}, axis='columns')
    return tabela_posthoc_final

In [56]:
for i in parametros:
    p = tabela_Anova_Final.loc["V5"+i][0]
    if p < 0.05:
        if math.isnan(p):
            print(i+' é nan')
        else:
            print('{} p = {}'.format(i,p))
            posthoc = FazerPostHocFinal(i)
            posthoc.to_excel('Dados_extras_posthoc_'+i+'_final.xlsx')


_PROT_G p = 0.001
_CARB_G p = 0.001
_SUGAR_G p = 0.001
_FAT_G p = 0.031
_SATFAT_G p = 0.032
_MONOFAT_G p = 0.003
_POLIFAT_G p = 0.03
_18_2-18_3 p = 0.037
_12_0_G p = 0.007
_14_0_G p = 0.0
_16_0_G p = 0.0
_18_0_G p = 0.037
_18_1_G p = 0.001
_18_2_G p = 0.042
_18_3_G p = 0.008
_20_5_G p = 0.001
_22_6_G p = 0.012
_IRON_MG p = 0.002
_SEL_MCG p = 0.001
_SOD_MG p = 0.003
_VIT_B6_MG p = 0.001
_VIT_B12_MCG p = 0.001
_FOL_ACID_MCG p = 0.002
_VIT_D_IU p = 0.003
_VIT_E_IU p = 0.001


In [57]:
resultado_final.query('GROUP==3')[["V0_VIT_A_IU","V5_VIT_A_IU"]]

,V0_VIT_A_IU,V5_VIT_A_IU
2,4476.285542,796.499731
5,2433.709498,434.450034
8,35856.046143,2664.498048
11,374.471962,2416.877898
14,2695.565838,1678.565024
17,1496.070417,12475.828006
20,3080.109209,4844.129288
